In [1]:
from pathlib import Path
import os
from febio_python.feb import FEBio_feb
import json

In [2]:
feb_template_path = Path("C:/Users/igorp/Downloads/LV_template_raw.feb")
json_data_path = Path("C:/Users/igorp/Downloads/lvtetmesh_wfibers_jsonsample.json")

In [3]:
with open(json_data_path, "r") as jfile:
  data = json.load(jfile)
data.keys()

dict_keys(['NODES', 'ELEMENTS', 'NODESETS', 'ELEMTSETS', 'SURFACES', 'VIRTUAL_NODES', 'DISCRETE_SETS', 'BOUNDARY_CONDITIONS', 'MESH_NODE_DATA', 'MESH_ELEM_DATA'])

In [4]:
feb = FEBio_feb.from_file(feb_template_path)
feb.add_nodes([
  {"name": "LV", 
   "nodes": data["NODES"]
   },
  {"name": "AORTIC_RIM", 
   "nodes": data["BOUNDARY_CONDITIONS"]["AORTIC_BC"][1]["RIM_NODES"]
   },
  {"name": "MITRAL_RIM", 
   "nodes": data["BOUNDARY_CONDITIONS"]["MITRAL_BC"][1]["RIM_NODES"]
   }
  ])
feb.add_elements([
  {
    "name": "LV", 
    "type": "tet4",
    "mat": "1",
    "elems": data["ELEMENTS"]["TETRA"]
    }
  ])
feb.add_nodesets(data["NODESETS"])
feb.add_surfaces(data["SURFACES"])
feb.add_discretesets(data["DISCRETE_SETS"])
feb.add_meshdata([
  {
    "elem_set": "LV", 
    "var": "mat_axis",
    "elems": {
      "a": data['MESH_ELEM_DATA']["LV_FIBERS"],
      "d": data['MESH_ELEM_DATA']["LV_SHEET"],
      }
    }
  ])

In [5]:
print(feb)

FEBio_feb:
-> Module: 0
-> Control: 19
-> Material: 2
--> material 'Holzapfel_Ogden_PAQ': 13
--> material 'rigid body': 2
-> Globals: 1
-> Geometry: 31
--> Nodes 'LV': 149636
--> Nodes 'AORTIC_RIM': 128
--> Nodes 'MITRAL_RIM': 128
--> Elements 'LV': 734930
-> Boundary: 5
-> Loads: 1
-> LoadData: 0
-> Output: 1
-> MeshData: 1



In [6]:
feb.write("./test.feb")